In [ ]:
import pandas as pd 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

df = pd.read_csv('https://assets.datacamp.com/production/repositories/6193/datasets/fca5067912db2b2346f568ce806915450fe56b99/water_potability.csv').to_numpy()
zero = df[100, :]

sample = torch.tensor([zero[:9]]).float()
target = torch.tensor([[1, 0]]).float()

weight = torch.randn(2, 9, requires_grad=True) 
bias = torch.randn(2, requires_grad=True)

preds = sample.matmul(weight.t()) + bias
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(preds, target)

# Compute the gradients of the loss
loss.backward()

# Display gradients of the weight and bias tensors in order
print(weight.grad)
print(bias.grad)

In [ ]:
y_hat = np.array(10)
y = np.array(1)

# Calculate the MSELoss using NumPy
mse_numpy = np.mean((y_hat-y)**2)

# Create the MSELoss function
criterion = nn.MSELoss()

# Calculate the MSELoss using the created loss function
mse_pytorch = criterion(torch.tensor(y_hat).float(), torch.tensor(y).float())
print(mse_pytorch)

In [ ]:
# Loop over the number of epochs and the dataloader
for i in range(num_epochs):
  for data in dataloader:
    # Set the gradients to zero
    optimizer.zero_grad()
    # Run a forward pass
    feature, target = data
    prediction = model(feature)    
    # Calculate the loss
    loss = criterion(prediction, target)    
    # Compute the gradients
    loss.backward()
    # Update the model's parameters
    optimizer.step()
show_results(model, dataloader)

In [ ]:
def calculate_capacity(model):
  total = 0
  for p in model.parameters():
    total += p.numel()
  return total

n_features = 8
n_classes = 2

input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Create a neural network with more than 120 parameters
model = nn.Sequential(nn.Linear(n_features, 12), nn.Linear(12,8), nn.Linear(8,4),
nn.Linear(4, n_classes))

output = model(input_tensor)

print(calculate_capacity(model))

In [ ]:
'''Manipulating the capacity of the network'''
n_features = 8
n_classes = 2

input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Create a neural network with less than 120 parameters
model = nn.Sequential(nn.Linear(n_features, 8),
nn.Linear(8, 4),
nn.Linear(4 ,n_classes))
output = model(input_tensor)

print(calculate_capacity(model))

In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

np_features = np.array(np.random.rand(12, 8))
np_target = np.array(np.random.rand(12, 1))

# Convert arrays to PyTorch tensors
torch_features = torch.tensor(np_features)
torch_target = torch.tensor(np_target)

# Create a TensorDataset from two tensors
dataset = TensorDataset(torch_features, torch_target)

# Return the last element of this dataset
print(dataset[-1])

In [ ]:
# Load the different columns into two PyTorch tensors
features = torch.tensor(np.array(
  dataframe[['ph', 'Sulfate', 'Conductivity', 'Organic_carbon']])).float()
target = torch.tensor(np.array(
  dataframe['Potability'])).float()

# Create a dataset from the two generated tensors
dataset = TensorDataset(features, target)

# Create a dataloader using the above dataset
dataloader = DataLoader(dataset, shuffle=True, batch_size=2)
x, y = next(iter(dataloader))

# Create a model using the nn.Sequential API
model = nn.Sequential(nn.Linear(4,2),nn.Linear(2,1))
output = model(features)
print(output)

In [ ]:
# Set the model to evaluation mode
model.eval()
validation_loss = 0.0

with torch.no_grad():
  
  for data in validationloader:
    
      outputs = model(data[0])
      loss = criterion(outputs, data[1])
      
      # Sum the current loss to the validation_loss variable
      validation_loss += loss.item()
      
# Calculate the mean loss value
validation_loss_epoch = validation_loss/len(validationloader)
print(validation_loss_epoch)

# Set the model back to training mode
model.train()

In [ ]:
# Create accuracy metric using torch metrics
metric = torchmetrics.Accuracy(task="multiclass", num_classes=3)
for data in dataloader:
    features, labels = data
    outputs = model(features)
    
    # Calculate accuracy over the batch
    acc = metric(outputs.softmax(dim=-1), labels.argmax(dim=-1))
    
# Calculate accuracy over the whole epoch
acc = metric.compute()

# Reset the metric for the next epoch 
metric.reset()
plot_errors(model, dataloader)

In [ ]:
'''implementing random search'''
values = []
for idx in range(10):
    # Randomly sample a learning rate factor between 0.01 and 0.0001
    factor = np.random.uniform(2,4)
    lr = 10 ** -factor
    
    # Randomly select a momentum between 0.85 and 0.99
    momentum = np.random.uniform(0.85, 0.99)
    
    values.append((lr, momentum))

In [ ]:
import numpy as np
from scipy.ndimage import convolve
from PIL import Image

class SIFTAlgorithm:
    def __init__(self):
        pass

    def gaussian_blur(self, img, sigma):
        size = int(2*(np.ceil(3*sigma))+1)
        x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
        g = np.exp(-(x**2 + y**2) / (2*sigma**2))
        g /= g.sum()
        img_blur = np.zeros_like(img)
        for i in range(3):  
            img_blur[:,:,  i] = convolve(img[:,:, i], g, mode='constant', cval=0)
        return img_blur

    def difference_of_gaussian(self, img1, img2):
        return img1 - img2

    def find_keypoints(self, dog_img, threshold=0.5):
        keypoints = np.argwhere(dog_img > threshold)
        return keypoints  # Return array of (row, col) positions

    def generate_descriptors(self, keypoints, img):
        descriptors = []
        for kp in keypoints:
            x, y = kp
            if x <= 0 or x >= img.shape[0]-1 or y <= 0 or y >= img.shape[1]-1:
                continue
            # Simple descriptor: use surrounding pixel values
            patch = img[x-1:x+2, y-1:y+2, :].flatten()
            descriptors.append(patch)
        return np.array(descriptors)

    def out(self, img):
        print(img.shape)
        h, w, c = img.shape
        img_gray = np.mean(img, axis=2)  

        img_blur1 = self.gaussian_blur(img_gray, sigma=1)
        img_blur2 = self.gaussian_blur(img_gray, sigma=2)
        dog_img = self.difference_of_gaussian(img_blur1, img_blur2)

        keypoints = self.find_keypoints(dog_img)

        descriptors = self.generate_descriptors(keypoints, img_gray)

        return keypoints, descriptors

sift = SIFTAlgorithm()
image = Image.open('11102.jpg')
img = np.asarray(image)
print(sift.out(img=img))